In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb
%run vis/helpersFunctions/barCharts.ipynb

Selectors :
   * xaxisCompletableTime : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisCompletableTime=widgets.Dropdown(options=[('All data',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
              description='x axis',
              disabled=False)

***vis_completables_times* function**:
* Output : the widgets selector and the outputCompletableTime in a list to be display in a VBox
* outputCompletableTime : a bar chart showing for each completable the maximum, minimum and mean time of completion by players

In [ ]:
outputCompletableTime=widgets.Output()
def vis_completables_times():

    def display_completable_time(change):
        with outputCompletableTime:
            clear_output(wait=False)
            construct_completable_time_bar_chart()
            clear_output(wait=True)
    xaxisCompletableTime.observe(display_completable_time,'value')
    display_completable_time(None)
    return [xaxisCompletableTime, outputCompletableTimeUpdate, outputCompletableTime]

***vis_completables_times* function**:
Output : Displays bar chart showing for each completable the maximum, minimum and mean time of completion by players

In [ ]:
def construct_completable_time_bar_chart():
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completablesSelectedCompletableTime=getSublistSelectedFromUnionListSelected(completables_times_list, checkboxesCompletableSelected.get_selected_options())
    if len(completablesSelectedCompletableTime)>0 and len(playersSelected)>0:
        if xaxisCompletableTime.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(completablesSelectedCompletableTime)/xaxisCompletableTime.value
            nbgraph=math.ceil(div)
        accordionCompletableTime=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionCompletableTime.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionCompletableTime)
        def on_selected_index_completable_score_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of completable time
                for fig in figCompletableTime:
                    plt.close(fig=fig)
                figCompletableTime.clear()
                #get all players selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                completablesSelectedCompletableTime=getSublistSelectedFromUnionListSelected(completables_times_list, checkboxesCompletableSelected.get_selected_options())
                # max, min time per completable
                max_times = []
                min_times = []
                for c in completablesSelectedCompletableTime:
                    maximum = 0
                    minimum = float('Inf')
                    for player in playersSelected:
                        if c in players_info[player]["completables_times"].keys():
                            times = players_info[player]["completables_times"][c]
                            try:
                                if len(times)==2:
                                    end_time = times[1]
                                    start_time = times[0]
                                    raw_total_time = end_time-start_time
                                    total_time = raw_total_time.days*24*60+raw_total_time.seconds/60.0 # minutes
                                    # update values
                                    if total_time > maximum:
                                        maximum = total_time
                                    if total_time < minimum:
                                        minimum = total_time
                            except:
                                pass
                    if minimum==float('Inf'):
                        minimum=maximum
                    max_times.append(maximum)
                    min_times.append(minimum)
                N=len(completablesSelectedCompletableTime)
                # separe in sub dict in function of xaxisCompletableTime dropout value selected
                if xaxisCompletableTime.value==0:  #getAllDataInAListOfList
                    completablesSelectedCompletableTime=[completablesSelectedCompletableTime]
                    min_times=[min_times]
                    max_times=[max_times]
                    nbgraphshown=1
                elif xaxisCompletableTime.value==-1:  #getnbFirstData
                    nb=10
                    nbgraphshown=1
                    completablesSelectedCompletableTime=[completablesSelectedCompletableTime[:nb]]
                    max_times=[max_times[:nb]]
                    min_times=[min_times[:nb]]
                    if N>nb:
                        N=nb
                elif xaxisCompletableTime.value==-2:  #getnbLastData
                    nb=10
                    nbgraphshown=1
                    completablesSelectedCompletableTime=[completablesSelectedCompletableTime[-nb:]]
                    max_times=[max_times[-nb:]]
                    min_times=[min_times[-nb:]]
                    if N>nb:
                        N=nb
                else:   #getxaxisDataSeparatedInListOfLists
                    completablesSelectedCompletableTime=separateListToSubListEachXvalues(completablesSelectedCompletableTime,xaxisCompletableTime.value)
                    max_times=separateListToSubListEachXvalues(max_times, xaxisCompletableTime.value)
                    min_times=separateListToSubListEachXvalues(min_times, xaxisCompletableTime.value)
                    N=xaxisCompletableTime.value
                    nbgraphshown=len(max_times)
                # Bar chart showing for each completable the maximum, minimum and mean time of completion by players.
                with accordionCompletableTime.children[n]:
                    #create new fig with custom name in the tab
                    name=filename+ "_completable_time"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig=plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figCompletableTime.append(fig)
                    if len(completablesSelectedCompletableTime)>0 and len(playersSelected)>0:
                        # display multibar min max
                        if n<nbgraphshown-1:
                            index = np.arange(N)
                        else:
                            index = np.arange(len(completablesSelectedCompletableTime[n]))
                        bar_width = 0.35
                        times=[max_times,min_times]
                        labels=['max', 'min']
                        max=0
                        for t in times[0][n]:
                            if t>max:
                                max=t
                        max+=5
                        for i in range(len(labels)):
                            # display bar
                            bar=plt.bar(index+i*bar_width, times[i][n], bar_width,
                                    label=labels[i],
                                    tick_label=completablesSelectedCompletableTime[n])
                            ticks_label=completablesSelectedCompletableTime[n]
                            add_value_labels(bar)
                        plt.ylim(0,max)
                        # display legend at a location
                        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
                        plt.gca().axes.set_xticks([x+bar_width/2 for x in range(len(ticks_label))])
                        plt.gca().axes.set_xticklabels(ticks_label)
                #add title and labels to graph
                plt.xlabel("Completable")
                plt.ylabel("Completion time (in minutes)")
                plt.xticks(rotation=90)
                plt.title("Max and min completion times per completable")
                plt.show()
        accordionCompletableTime.observe(on_selected_index_completable_score_change,'selected_index')
        accordionCompletableTime.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=filename+ "_completable_time"
        fig=plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figCompletableTime.append(fig)
        #display no data to fill vis
        noDataToFillVis(100)
        #add title and labels to graph
        plt.xlabel("Completable")
        plt.ylabel("Completion time (in minutes)")
        plt.xticks(rotation=90)
        plt.title("Max and min completion times per completable")
        plt.show()
    with outputCompletableTimeUpdate:
        clear_output(wait=False)